# Financial ML System - Backtesting

This notebook evaluates the trading strategy performance.

## 1. Setup

In [ ]:
import sys
import warnings
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')

PROJECT_ROOT = Path('/content/financial-ml-system')
sys.path.insert(0, str(PROJECT_ROOT))

from src.utils.constants import DATA_DIR, TRADING_DAYS_PER_YEAR, RISK_FREE_RATE, SIGNAL_BUY, SIGNAL_SELL
from src.utils.helpers import calculate_sharpe_ratio, calculate_max_drawdown

print("Setup complete")

## 2. Load Signals

In [ ]:
ticker = 'SPY'
signals_file = DATA_DIR / 'processed' / f"{ticker}_signals.csv"

data = pd.read_csv(signals_file, index_col=0, parse_dates=True)
print(f"Loaded {len(data)} rows")
data.head()

## 3. Backtesting Engine

In [ ]:
class Backtester:
    """Backtest trading strategy."""
    
    def __init__(self, initial_capital=10000, commission=0.001, slippage=0.0005):
        self.initial_capital = initial_capital
        self.commission = commission
        self.slippage = slippage
    
    def run(self, data):
        """Run backtest."""
        portfolio = pd.DataFrame(index=data.index)
        portfolio['Signal'] = data['Signal']
        portfolio['Position_Size'] = data['Position_Size']
        portfolio['Price'] = data['Close']
        portfolio['Returns'] = data['Returns']
        
        # Initialize
        portfolio['Position'] = 0.0
        portfolio['Cash'] = self.initial_capital
        portfolio['Holdings'] = 0.0
        portfolio['Total'] = self.initial_capital
        portfolio['Trades'] = 0
        
        cash = self.initial_capital
        position = 0.0
        trades = 0
        
        for i in range(1, len(portfolio)):
            signal = portfolio['Signal'].iloc[i]
            pos_size = portfolio['Position_Size'].iloc[i]
            price = portfolio['Price'].iloc[i]
            
            # Execute trades
            if signal == SIGNAL_BUY and position <= 0:
                # Buy
                target_value = self.initial_capital * pos_size
                shares_to_buy = target_value / (price * (1 + self.commission + self.slippage))
                cost = shares_to_buy * price * (1 + self.commission + self.slippage)
                
                if cost <= cash:
                    position = shares_to_buy
                    cash -= cost
                    trades += 1
            
            elif signal == SIGNAL_SELL and position >= 0:
                # Sell (or short)
                if position > 0:
                    # Close long position
                    proceeds = position * price * (1 - self.commission - self.slippage)
                    cash += proceeds
                    position = 0
                    trades += 1
            
            # Update portfolio
            portfolio.loc[portfolio.index[i], 'Position'] = position
            portfolio.loc[portfolio.index[i], 'Cash'] = cash
            portfolio.loc[portfolio.index[i], 'Holdings'] = position * price
            portfolio.loc[portfolio.index[i], 'Total'] = cash + (position * price)
            portfolio.loc[portfolio.index[i], 'Trades'] = trades
        
        return portfolio
    
    @staticmethod
    def calculate_metrics(portfolio):
        """Calculate performance metrics."""
        returns = portfolio['Total'].pct_change().dropna()
        
        metrics = {
            'Total Return': (portfolio['Total'].iloc[-1] / portfolio['Total'].iloc[0] - 1) * 100,
            'Annual Return': returns.mean() * TRADING_DAYS_PER_YEAR * 100,
            'Annual Volatility': returns.std() * np.sqrt(TRADING_DAYS_PER_YEAR) * 100,
            'Sharpe Ratio': calculate_sharpe_ratio(returns, RISK_FREE_RATE, TRADING_DAYS_PER_YEAR),
            'Max Drawdown': calculate_max_drawdown(portfolio['Total']) * 100,
            'Win Rate': (returns > 0).sum() / len(returns) * 100,
            'Total Trades': int(portfolio['Trades'].iloc[-1]),
            'Final Value': portfolio['Total'].iloc[-1]
        }
        
        return metrics

print("Backtester class defined")

## 4. Run Backtest

In [ ]:
# Initialize backtester
backtester = Backtester(initial_capital=10000)

# Run strategy backtest
portfolio = backtester.run(data)

# Calculate metrics
metrics = backtester.calculate_metrics(portfolio)

print("Strategy Performance")
print("=" * 50)
for metric, value in metrics.items():
    if 'Return' in metric or 'Volatility' in metric or 'Drawdown' in metric or 'Rate' in metric:
        print(f"{metric:<25} {value:>10.2f}%")
    elif 'Ratio' in metric:
        print(f"{metric:<25} {value:>10.2f}")
    elif 'Value' in metric:
        print(f"{metric:<25} ${value:>10,.2f}")
    else:
        print(f"{metric:<25} {value:>10}")

## 5. Buy and Hold Comparison

In [ ]:
# Buy and hold strategy
initial_price = data['Close'].iloc[0]
shares = backtester.initial_capital / initial_price
buy_hold_value = shares * data['Close']
buy_hold_returns = buy_hold_value.pct_change().dropna()

buy_hold_metrics = {
    'Total Return': (buy_hold_value.iloc[-1] / buy_hold_value.iloc[0] - 1) * 100,
    'Annual Return': buy_hold_returns.mean() * TRADING_DAYS_PER_YEAR * 100,
    'Annual Volatility': buy_hold_returns.std() * np.sqrt(TRADING_DAYS_PER_YEAR) * 100,
    'Sharpe Ratio': calculate_sharpe_ratio(buy_hold_returns, RISK_FREE_RATE, TRADING_DAYS_PER_YEAR),
    'Max Drawdown': calculate_max_drawdown(buy_hold_value) * 100,
    'Win Rate': (buy_hold_returns > 0).sum() / len(buy_hold_returns) * 100,
    'Total Trades': 1,
    'Final Value': buy_hold_value.iloc[-1]
}

print("\nBuy & Hold Performance")
print("=" * 50)
for metric, value in buy_hold_metrics.items():
    if 'Return' in metric or 'Volatility' in metric or 'Drawdown' in metric or 'Rate' in metric:
        print(f"{metric:<25} {value:>10.2f}%")
    elif 'Ratio' in metric:
        print(f"{metric:<25} {value:>10.2f}")
    elif 'Value' in metric:
        print(f"{metric:<25} ${value:>10,.2f}")
    else:
        print(f"{metric:<25} {value:>10}")

## 6. Comparison Table

In [ ]:
# Comparison
comparison = pd.DataFrame({
    'Strategy': metrics,
    'Buy & Hold': buy_hold_metrics,
    'Difference': {k: metrics[k] - buy_hold_metrics[k] for k in metrics.keys()}
})

print("\nStrategy vs Buy & Hold")
print("=" * 70)
print(comparison.to_string())

## 7. Portfolio Value Chart

In [ ]:
fig, ax = plt.subplots(figsize=(14, 7))

# Strategy portfolio
ax.plot(portfolio.index, portfolio['Total'], label='ML Strategy', linewidth=2)

# Buy and hold
ax.plot(data.index, buy_hold_value, label='Buy & Hold', linewidth=2, linestyle='--', alpha=0.7)

ax.set_title('Portfolio Value Over Time', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Portfolio Value ($)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'portfolio_value.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Drawdown Analysis

In [ ]:
# Calculate drawdowns
def calculate_drawdown_series(values):
    cumulative = values
    running_max = cumulative.expanding().max()
    drawdown = (cumulative - running_max) / running_max
    return drawdown

strategy_dd = calculate_drawdown_series(portfolio['Total'])
buyhold_dd = calculate_drawdown_series(buy_hold_value)

fig, ax = plt.subplots(figsize=(14, 7))

ax.fill_between(portfolio.index, 0, strategy_dd * 100, alpha=0.3, label='ML Strategy')
ax.fill_between(data.index, 0, buyhold_dd * 100, alpha=0.3, label='Buy & Hold')

ax.set_title('Drawdown Comparison', fontsize=14, fontweight='bold')
ax.set_xlabel('Date')
ax.set_ylabel('Drawdown (%)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'drawdown.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. Returns Distribution

In [ ]:
strategy_returns = portfolio['Total'].pct_change().dropna()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
ax1.hist(strategy_returns * 100, bins=50, alpha=0.7, edgecolor='black', label='Strategy')
ax1.hist(buy_hold_returns * 100, bins=50, alpha=0.5, edgecolor='black', label='Buy & Hold')
ax1.axvline(x=0, color='red', linestyle='--', linewidth=2)
ax1.set_title('Daily Returns Distribution', fontsize=14, fontweight='bold')
ax1.set_xlabel('Returns (%)')
ax1.set_ylabel('Frequency')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Cumulative returns
cumulative_strategy = (1 + strategy_returns).cumprod()
cumulative_buyhold = (1 + buy_hold_returns).cumprod()

ax2.plot(portfolio.index[1:], cumulative_strategy, label='Strategy', linewidth=2)
ax2.plot(data.index[1:], cumulative_buyhold, label='Buy & Hold', linewidth=2, linestyle='--')
ax2.set_title('Cumulative Returns', fontsize=14, fontweight='bold')
ax2.set_xlabel('Date')
ax2.set_ylabel('Cumulative Return')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(PROJECT_ROOT / 'results' / 'returns_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 10. Trade Analysis

In [ ]:
# Analyze trades
trade_changes = portfolio['Trades'].diff()
trade_dates = portfolio[trade_changes > 0].index

print(f"\nTotal Trades: {int(portfolio['Trades'].iloc[-1])}")
print(f"Average holding period: {len(portfolio) / max(portfolio['Trades'].iloc[-1], 1):.1f} days")

if len(trade_dates) > 0:
    print(f"\nFirst 10 trades:")
    for i, date in enumerate(trade_dates[:10]):
        signal = portfolio.loc[date, 'Signal']
        price = portfolio.loc[date, 'Price']
        print(f"{i+1}. {date.date()}: {signal} at ${price:.2f}")

## 11. Save Results

In [ ]:
import json

# Save results
results = {
    'strategy_metrics': metrics,
    'buyhold_metrics': buy_hold_metrics,
    'comparison': comparison.to_dict()
}

results_file = PROJECT_ROOT / 'results' / 'backtest_results.json'
with open(results_file, 'w') as f:
    json.dump(results, f, indent=2)

# Save portfolio
portfolio_file = PROJECT_ROOT / 'results' / 'portfolio_history.csv'
portfolio.to_csv(portfolio_file)

print(f"Results saved to: {results_file}")
print(f"Portfolio history saved to: {portfolio_file}")

## 12. Final Summary

In [ ]:
print("\n" + "=" * 70)
print("FINAL SUMMARY".center(70))
print("=" * 70)

print(f"\nStrategy outperformed Buy & Hold: {metrics['Total Return'] > buy_hold_metrics['Total Return']}")
print(f"Return difference: {metrics['Total Return'] - buy_hold_metrics['Total Return']:.2f}%")
print(f"Sharpe ratio improvement: {metrics['Sharpe Ratio'] - buy_hold_metrics['Sharpe Ratio']:.2f}")
print(f"Drawdown improvement: {buy_hold_metrics['Max Drawdown'] - metrics['Max Drawdown']:.2f}%")

print("\n" + "=" * 70)
print("All notebooks complete!")
print("Check the results/ folder for charts and analysis.")
print("=" * 70)

## Project Complete!

You have successfully:
1. Set up the project structure
2. Ingested and cleaned market data
3. Engineered technical features
4. Trained SVM classifier for regime detection
5. Trained neural network for return prediction
6. Generated trading signals
7. Backtested the strategy

Next steps:
- Experiment with different parameters
- Try different tickers
- Add more features
- Implement portfolio optimization
- Deploy for live trading (paper trading first!)